In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
import numpy as np
from fake_useragent import UserAgent
import random

In [2]:
ua = UserAgent()
user_agent = {'User-Agent': ua.random}

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/opt/anaconda3/envs/metis/lib/python3.8/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/opt/anaconda3/envs/metis/lib/python3.8/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [3]:
ticker_dict = {'equity':[],
               'publishing_analysts':[],
               'enterprise_value':[],
               'market_cap':[],
               'avg_vol':[],
               'bso':[],
               'float_shares':[],
               'roa':[],
               'roe':[],
               'ltm_rev':[],
               'gross_profit':[],
               'operating_margin':[],
               'ltm_ebitda':[],
               'mrq_assets':[]
              }

In [4]:
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN

def get_ticker_data(ticker):
    
    temp_dict = {}
    try:
        # set url paths and BS pages based on input ticker
        statistics_url = f'https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}'
        analysis_url = f'https://finance.yahoo.com/quote/{ticker}/analysis?p={ticker}'
        bs_url = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
                     
        statistics_page = BeautifulSoup(requests.get(statistics_url,headers=user_agent).text,'html.parser')
        bs_page = BeautifulSoup(requests.get(bs_url,headers=user_agent).text,'html.parser')
        analysis_page = BeautifulSoup(requests.get(analysis_url,headers=user_agent).text,'html.parser')
        
        
        # get publishing analyst count
        publishing_analysts = analysis_page.find_all('span',class_='Trsdu(0.3s)')[16].text
        
        # get enterprise value
        enterprise_value = statistics_page.find_all('td')[8].text
        
        # get market cap
        market_cap = statistics_page.find_all('td')[1].text
        
        # get avg volume
        avg_vol = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdY Bdc($seperatorColor)')[1].text
        
        # get basic shares outstanding
        bso = statistics_page.find_all('td',class_='Fw(500) Ta(end) Pstart(10px) Miw(60px)')[8].text
        
        # get float shares
        float_shares = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdB Bdbc($seperatorColor)')[9].text
        
        # get roa
        roa = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdY Bdc($seperatorColor)')[5].text
        
        # get roe
        roe = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdB Bdbc($seperatorColor)')[28].text
        
        # get ltm revenue
        ltm_rev = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdY Bdc($seperatorColor)')[6].text
        
        # get gross profit dollars
        gross_profit = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdB Bdbc($seperatorColor)')[31].text
        
        # get operating profit margin
        operating_margin = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdB Bdbc($seperatorColor)')[27].text
        
        # get ltm EBITDA
        ltm_ebitda = statistics_page.find_all('tr',class_='Bxz(bb) H(36px) BdB Bdbc($seperatorColor)')[32].text
        
        # get mrq assets book value
        mrq_assets = bs_page.find_all('div',class_='Ta(c) Py(6px) Bxz(bb) BdB Bdc($seperatorColor) Miw(120px) Miw(140px)--pnclg D(tbc)')[0].text
      
        temp_dict[ticker] = [publishing_analysts,enterprise_value,market_cap,avg_vol,
                               bso,float_shares,roa,roe,ltm_rev,gross_profit,
                               operating_margin,ltm_ebitda,mrq_assets]
    
    except IndexError:
        temp_dict[ticker] = [np.nan,np.nan,np.nan,np.nan,
                               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                               np.nan,np.nan,np.nan]
        
    return temp_dict

In [6]:
# pull Russell 1000 tickers from csv

russ_df = pd.read_csv(r'/Users/sam/Desktop/russ_1000.csv')
ticker_list = list(russ_df.Symbol)

In [7]:
# add ticker info to dictionary for dataframe
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN
# THIS CAN TAKE >8 HOURS TO RUN

ticker_dict = {}
for i in ticker_list:    
    ticker_dict.update(get_ticker_data(i))
    time.sleep(25 + 5*random.random())

In [136]:
# create dataframe
df = pd.DataFrame(ticker_dict).T
df.reset_index(inplace=True)
df.columns = ['equity','publishing_analysts','enterprise_value','market_cap','avg_vol',
              'bso','float_shares','roa','roe','ltm_rev','gross_profit',
              'operating_margin','ltm_ebitda','mrq_assets']
df.dropna(inplace=True)



In [137]:
# export to csv
df.to_csv(r'/Users/sam/Desktop/raw_ticker_df.csv')